# CONTENT BASED RECCOMENDATION SYSTEM (TF-IDF)

In [1]:
import numpy as np
import pandas as pd

In [2]:
credits = pd.read_csv("credits.csv")
keywords = pd.read_csv("keywords.csv")
movies_metadata= pd.read_csv("movies_metadata.csv")
#movie = pd.read_csv(r"C:\Users\mirza\Desktop\dataset\MovieLens-Credits\ratings.csv")

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\3452722229.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata= pd.read_csv("movies_metadata.csv")


## Data Preprocessing


In [3]:
movies_meta = movies_metadata[movies_metadata["id"].apply(lambda x: x.isnumeric())]
movies_meta["id"]= movies_meta["id"].astype(int)

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\1325305789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_meta["id"]= movies_meta["id"].astype(int)


In [4]:
merged_dataset= pd.merge(pd.merge( movies_meta, credits, how="inner", on = "id"), keywords, how="inner", on = "id")

In [5]:
merged_dataset.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [6]:
#merged_dataset= pd.merge(pd.merge(movies_meta, credits , how="inner", on = "id"), keywords, how="inner", on = "id")

In [7]:
movies = merged_dataset[["id", "title", "overview", "genres", "keywords", "cast", "crew"]] 

In [8]:
movies.isnull().sum()

id            0
title         4
overview    995
genres        0
keywords      0
cast          0
crew          0
dtype: int64

In [9]:
movies.dropna(inplace=True)

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\3786870272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(inplace=True)


In [10]:
movies.iloc[0].genres

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [11]:
# I need only names, that is why ı will create a functıon to extract only names.

# "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

# to 

#["Drama", "Mystery", "Romance"]

import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i["name"])
    
    return L
        

In [12]:
movies["genres"]= movies["genres"].apply(convert)

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\3357794645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["genres"]= movies["genres"].apply(convert)


In [13]:
# ı will do same thing for Keywords column
movies["keywords"]= movies["keywords"].apply(convert)

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\2267770242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["keywords"]= movies["keywords"].apply(convert)


In [14]:
movies.iloc[0].genres

['Animation', 'Comedy', 'Family']

In [15]:
movies.iloc[0].cast

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [16]:
# I need only casts names, that is why ı will create a functıon to extract only names.

# to 

# ['Tom Hanks', 'Tim Allen', 'Don Rickles']

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i["name"])
            counter+=1
        else:
            break
    
    return L
        

In [17]:
movies["cast"]=movies["cast"].apply(convert3)

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\2992349304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["cast"]=movies["cast"].apply(convert3)


In [18]:
movies.iloc[0].cast

['Tom Hanks', 'Tim Allen', 'Don Rickles']

In [19]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]","[jealousy, toy, boy, friendship, friends, riva...","[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]","[board game, disappearance, based on children'...","[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[fishing, best friend, duringcreditsstinger, o...","[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[based on novel, interracial relationship, sin...","[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[baby, midlife crisis, confidence, aging, daug...","[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [20]:
movies["crew"][0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [21]:
# I want to extract  director names

def fetch_director(obj):
    L = []
    for i in ast.literal_eval((obj)):
        if i["job"] == "Director":
            L.append(i["name"])
            break
    
    return L

In [22]:
movies["crew"]=movies["crew"].apply(fetch_director)

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\2208503418.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["crew"]=movies["crew"].apply(fetch_director)


In [23]:
movies["crew"][0]

['John Lasseter']

In [24]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]","[jealousy, toy, boy, friendship, friends, riva...","[Tom Hanks, Tim Allen, Don Rickles]",[John Lasseter]
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]","[board game, disappearance, based on children'...","[Robin Williams, Jonathan Hyde, Kirsten Dunst]",[Joe Johnston]
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[fishing, best friend, duringcreditsstinger, o...","[Walter Matthau, Jack Lemmon, Ann-Margret]",[Howard Deutch]
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[based on novel, interracial relationship, sin...","[Whitney Houston, Angela Bassett, Loretta Devine]",[Forest Whitaker]
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[baby, midlife crisis, confidence, aging, daug...","[Steve Martin, Diane Keaton, Martin Short]",[Charles Shyer]


In [25]:
movies["overview"][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [26]:
movies["overview"]= movies["overview"].apply(lambda x:x.split())
# I split the per words on overview for vectorization

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\1440631771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["overview"]= movies["overview"].apply(lambda x:x.split())


In [27]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,862,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,...","[Animation, Comedy, Family]","[jealousy, toy, boy, friendship, friends, riva...","[Tom Hanks, Tim Allen, Don Rickles]",[John Lasseter]
1,8844,Jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[Adventure, Fantasy, Family]","[board game, disappearance, based on children'...","[Robin Williams, Jonathan Hyde, Kirsten Dunst]",[Joe Johnston]
2,15602,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ...","[Romance, Comedy]","[fishing, best friend, duringcreditsstinger, o...","[Walter Matthau, Jack Lemmon, Ann-Margret]",[Howard Deutch]
3,31357,Waiting to Exhale,"[Cheated, on,, mistreated, and, stepped, on,, ...","[Comedy, Drama, Romance]","[based on novel, interracial relationship, sin...","[Whitney Houston, Angela Bassett, Loretta Devine]",[Forest Whitaker]
4,11862,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr...",[Comedy],"[baby, midlife crisis, confidence, aging, daug...","[Steve Martin, Diane Keaton, Martin Short]",[Charles Shyer]


In [28]:
# I split with comma the per words on overview for vectorization

movies["genres"] = movies["genres"].apply(lambda x:[i.replace(" ", "") for i in x ])
movies["keywords"] = movies["keywords"].apply(lambda x:[i.replace(" ", "") for i in x ])
movies["cast"] = movies["cast"].apply(lambda x:[i.replace(" ", "") for i in x ])
movies["crew"] = movies["crew"].apply(lambda x:[i.replace(" ", "") for i in x ])

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\2867527388.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["genres"] = movies["genres"].apply(lambda x:[i.replace(" ", "") for i in x ])
C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\2867527388.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["keywords"] = movies["keywords"].apply(lambda x:[i.replace(" ", "") for i in x ])
C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\2867527388.py:5: SettingWithCopyWarning: 
A value is trying to

In [29]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,862,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,...","[Animation, Comedy, Family]","[jealousy, toy, boy, friendship, friends, riva...","[TomHanks, TimAllen, DonRickles]",[JohnLasseter]
1,8844,Jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[Adventure, Fantasy, Family]","[boardgame, disappearance, basedonchildren'sbo...","[RobinWilliams, JonathanHyde, KirstenDunst]",[JoeJohnston]
2,15602,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ...","[Romance, Comedy]","[fishing, bestfriend, duringcreditsstinger, ol...","[WalterMatthau, JackLemmon, Ann-Margret]",[HowardDeutch]
3,31357,Waiting to Exhale,"[Cheated, on,, mistreated, and, stepped, on,, ...","[Comedy, Drama, Romance]","[basedonnovel, interracialrelationship, single...","[WhitneyHouston, AngelaBassett, LorettaDevine]",[ForestWhitaker]
4,11862,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr...",[Comedy],"[baby, midlifecrisis, confidence, aging, daugh...","[SteveMartin, DianeKeaton, MartinShort]",[CharlesShyer]


In [30]:
# I gather on  same columns all ınformatıons about the movies for vectorization

movies["tags"]= movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\1444024438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["tags"]= movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]


In [31]:
movies.head()

,id,title,overview,genres,keywords,cast,crew,tags
0,862,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,...","[Animation, Comedy, Family]","[jealousy, toy, boy, friendship, friends, riva...","[TomHanks, TimAllen, DonRickles]",[JohnLasseter],"[Led, by, Woody,, Andy's, toys, live, happily,..."
1,8844,Jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[Adventure, Fantasy, Family]","[boardgame, disappearance, basedonchildren'sbo...","[RobinWilliams, JonathanHyde, KirstenDunst]",[JoeJohnston],"[When, siblings, Judy, and, Peter, discover, a..."
2,15602,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ...","[Romance, Comedy]","[fishing, bestfriend, duringcreditsstinger, ol...","[WalterMatthau, JackLemmon, Ann-Margret]",[HowardDeutch],"[A, family, wedding, reignites, the, ancient, ..."
3,31357,Waiting to Exhale,"[Cheated, on,, mistreated, and, stepped, on,, ...","[Comedy, Drama, Romance]","[basedonnovel, interracialrelationship, single...","[WhitneyHouston, AngelaBassett, LorettaDevine]",[ForestWhitaker],"[Cheated, on,, mistreated, and, stepped, on,, ..."
4,11862,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr...",[Comedy],"[baby, midlifecrisis, confidence, aging, daugh...","[SteveMartin, DianeKeaton, MartinShort]",[CharlesShyer],"[Just, when, George, Banks, has, recovered, fr..."


In [32]:
new_df = movies[["id", "title", "tags"]]

In [33]:
new_df

,id,title,tags
0,862,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,..."
1,8844,Jumanji,"[When, siblings, Judy, and, Peter, discover, a..."
2,15602,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ..."
3,31357,Waiting to Exhale,"[Cheated, on,, mistreated, and, stepped, on,, ..."
4,11862,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr..."
...,...,...,...
46623,439050,Subdue,"[Rising, and, falling, between, a, man, and, w..."
46624,111109,Century of Birthing,"[An, artist, struggles, to, finish, his, work,..."
46625,67758,Betrayal,"[When, one, of, her, hits, goes, wrong,, a, pr..."
46626,227506,Satan Triumphant,"[In, a, small, town, live, two, brothers,, one..."


In [34]:
new_df["tags"][0]

['Led',
 'by',
 'Woody,',
 "Andy's",
 'toys',
 'live',
 'happily',
 'in',
 'his',
 'room',
 'until',
 "Andy's",
 'birthday',
 'brings',
 'Buzz',
 'Lightyear',
 'onto',
 'the',
 'scene.',
 'Afraid',
 'of',
 'losing',
 'his',
 'place',
 'in',
 "Andy's",
 'heart,',
 'Woody',
 'plots',
 'against',
 'Buzz.',
 'But',
 'when',
 'circumstances',
 'separate',
 'Buzz',
 'and',
 'Woody',
 'from',
 'their',
 'owner,',
 'the',
 'duo',
 'eventually',
 'learns',
 'to',
 'put',
 'aside',
 'their',
 'differences.',
 'Animation',
 'Comedy',
 'Family',
 'jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boynextdoor',
 'newtoy',
 'toycomestolife',
 'TomHanks',
 'TimAllen',
 'DonRickles',
 'JohnLasseter']

In [35]:
new_df["tags"] = new_df["tags"].apply(lambda x :" ".join(x))

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\4169619949.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda x :" ".join(x))


In [36]:
new_df["tags"]= new_df["tags"].apply(lambda x:x.lower())

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\1634414329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]= new_df["tags"].apply(lambda x:x.lower())


In [37]:
new_df["tags"][0]

"led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences. animation comedy family jealousy toy boy friendship friends rivalry boynextdoor newtoy toycomestolife tomhanks timallen donrickles johnlasseter"

In [38]:
new_df

,id,title,tags
0,862,Toy Story,"led by woody, andy's toys live happily in his ..."
1,8844,Jumanji,when siblings judy and peter discover an encha...
2,15602,Grumpier Old Men,a family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,just when george banks has recovered from his ...
...,...,...,...
46623,439050,Subdue,rising and falling between a man and woman. dr...
46624,111109,Century of Birthing,an artist struggles to finish his work while a...
46625,67758,Betrayal,"when one of her hits goes wrong, a professiona..."
46626,227506,Satan Triumphant,"in a small town live two brothers, one a minis..."


In [39]:
import nltk
from nltk.stem.porter import PorterStemmer

# Its for catch  words of same meaning
ps = PorterStemmer()

In [40]:
#lovedi loving, love
ps.stem("loved")

'love'

In [41]:
def stem(text):
    y= []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [42]:
new_df["tags"]= new_df["tags"].apply(stem)

C:\Users\mirza\AppData\Local\Temp\ipykernel_10180\447794818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]= new_df["tags"].apply(stem)


In [43]:
new_df

,id,title,tags
0,862,Toy Story,"led by woody, andy' toy live happili in hi roo..."
1,8844,Jumanji,when sibl judi and peter discov an enchant boa...
2,15602,Grumpier Old Men,a famili wed reignit the ancient feud between ...
3,31357,Waiting to Exhale,"cheat on, mistreat and step on, the women are ..."
4,11862,Father of the Bride Part II,just when georg bank ha recov from hi daughter...
...,...,...,...
46623,439050,Subdue,rise and fall between a man and woman. drama f...
46624,111109,Century of Birthing,an artist struggl to finish hi work while a st...
46625,67758,Betrayal,"when one of her hit goe wrong, a profession as..."
46626,227506,Satan Triumphant,"in a small town live two brothers, one a minis..."


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
# We Vectorize the tags with 5000 features
cv = CountVectorizer(max_features=5000, stop_words = "english")

In [45]:
vectors = cv.fit_transform(new_df["tags"]).toarray()

In [46]:
vectors.shape

(45629, 5000)

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
# Find similar movies with cosine similarity

similaritiy = cosine_similarity(vectors)

In [49]:
similaritiy[0]

array([1.        , 0.07216878, 0.05025189, ..., 0.        , 0.17099639,
       0.        ])

In [50]:
 # Creating Reccomend functıon

def reccomend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similaritiy[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)


In [51]:
movie_index = new_df[new_df['title'] == "Jumanji"].index[0]
distances = similaritiy[movie_index]
movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

In [52]:
movies_list

[(21671, 0.3602883460614461),
 (42185, 0.3484365738222833),
 (44589, 0.3404414533245951),
 (30113, 0.2857738033247041),
 (35950, 0.28444006199428723)]

In [53]:
reccomend("Jumanji")

Table No. 21
Enter the Battlefield: Life on the Magic - The Gathering Pro Tour
Liar Game: Reborn
The Games Maker
100 Yen: The Japanese Arcade Experience


In [54]:
import pickle

In [55]:
pickle.dump(similaritiy, open("similaritiy.pkl","wb" ))
pickle.dump(new_df, open("movie_dict.pkl","wb" ))